# Logic to pull additional details from the JSON files without running the full original parsing

In [140]:
import nbformat
import pandas as pd
import numpy as np
import json
import os

pd.set_option('display.max_columns', None)

In [146]:
directory = ('../raw_data/plate_app_data/')
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    
    print(f)

../raw_data/plate_app_data/e60d1770-2789-4d00-b94a-5033a7fa89a9.json
../raw_data/plate_app_data/5367bb21-dfd1-42ea-8888-970f0d615fb9.json
../raw_data/plate_app_data/5afd98c8-3dfa-42f7-817a-c1812b67aa4b.json
../raw_data/plate_app_data/b0ef5568-1b76-478a-a85f-bb56119b9ecf.json
../raw_data/plate_app_data/5783e744-ad40-45b1-ac21-88bf68b106b8.json
../raw_data/plate_app_data/ec316169-965e-49a9-9e68-91483895ff27.json
../raw_data/plate_app_data/96685357-ff56-4b07-b6e0-9b15a958122c.json
../raw_data/plate_app_data/d6dc5082-8a6f-4895-88cd-ba2d5b592088.json
../raw_data/plate_app_data/89e40113-3ab5-477e-9591-93a6d40e174d.json
../raw_data/plate_app_data/98d8ec35-17ec-4fa2-878b-8a710867ece3.json
../raw_data/plate_app_data/d9517e7b-16c7-4b3d-af70-216493655ddf.json
../raw_data/plate_app_data/32ac90c3-c9a8-45e3-9b64-0e6487e42356.json
../raw_data/plate_app_data/41c5b41e-5aa1-468c-81a6-4fb1af840d02.json
../raw_data/plate_app_data/90ed3ca6-250e-4125-b878-1e00a8c2fa73.json
../raw_data/plate_app_data/acd8006

In [145]:
len(directory)

27

In [122]:
columns = ['id', 'at_bat_end_time', 'pitch_type_code', 'pitch_type_des', 'pitch_speed_mph', 'pitch_count_at_bat', 'pitcher_pitch_count_at_bat_start', 'outs_at_start', 'output_code']

In [156]:
new_df = pd.DataFrame(columns=columns)
new_df

,id,at_bat_end_time,pitch_type_code,pitch_type_des,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code


In [164]:
directory = ('../raw_data/plate_app_data/')
runs = 0

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    with open(f) as user_file:
        gdata = user_file.read()
    gdata = json.loads(gdata)

    for ing in range(1, len(gdata["game"]["innings"])):
        for hlf in range(2):
            for atb in range(len(gdata["game"]["innings"][ing]["halfs"][hlf]["events"])):
                
                if 'at_bat' not in gdata["game"]["innings"][ing]["halfs"][hlf]["events"][atb]:
                    continue
                
                id = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["id"]
                
                try: 
                    at_bat_end_time = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["wall_clock"]["end_time"]
                except: at_bat_end_time = np.nan
                
                try:
                    pitch_type_code = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["mlb_pitch_data"]["code"]
                except:
                    pitch_type_code = np.nan
                    
                try:
                    pitch_type_des = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["mlb_pitch_data"]["description"]
                except:
                    pitch_type_des = np.nan
                    
                try:
                    pitch_speed_mph = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["pitcher"]["pitch_speed"]
                except:
                    pitch_speed_mph = np.nan
                
                try:
                    pitch_count_at_bat = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["count"]["pitch_count"]
                except:
                    pitch_count_at_bat = np.nan
                
                try:    
                    pitcher_pitch_count_at_bat_start = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["pitcher"]["pitch_count"] - pitch_count_at_bat
                except:
                    pitcher_pitch_count_at_bat_start = np.nan
                    
                try:    
                    if atb == 0:
                        outs_at_start = 0
                    elif pitch_count_at_bat == 1:
                        outs_at_start = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb-1]["at_bat"]["events"][-1]["count"]["outs"]
                    else:
                        outs_at_start = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][0]["count"]["outs"]
                
                except:
                    outs_at_start = np.nan
                    
                try:
                    output_code = gdata["game"]['innings'][ing]['halfs'][hlf]['events'][atb]["at_bat"]["events"][-1]["outcome_id"]
                except:
                    output_code = np.nan
                    
                _temp = pd.DataFrame(list((id, at_bat_end_time, pitch_type_code, pitch_type_des, pitch_speed_mph, pitch_count_at_bat, pitcher_pitch_count_at_bat_start, outs_at_start, output_code))).T
                
                _temp.columns = columns

                new_df = pd.concat((new_df, _temp))
    runs += 1
    print(runs)

# new_df = new_df.set_index("id")         

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [173]:
add_df = new_df.drop_duplicates()

In [174]:
add_df.duplicated().sum()

0

In [175]:
add_df.describe()

,id,at_bat_end_time,pitch_type_code,pitch_type_des,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code
count,143424,143355,142683,142683,142652.0,143377,143271,141753,143271
unique,143423,140574,16,16,579.0,17,123,4,50
top,7b207fc7-c3f8-4513-94f4-ff57a4a57af6,2023-07-09T18:47:11+00:00,FF,Four-Seam Fastball,93.9,4,0,0,oGO
freq,2,3,44085,44085,1174.0,26827,15790,48543,26998


In [177]:
add_df.to_csv('../raw_data/all_ab_raw_data_add_columns.csv', index=True)